<a href="https://colab.research.google.com/github/Ardhy2357/Carbon-Data-Visualization/blob/main/Carbon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SETUP**

In [8]:
##setup
!pip install openpyxl

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Load Data**

In [ ]:
## Load excel or data
rawdata = pd.read_excel("/content/drive/MyDrive/Data Vis/mock_carbon_data.xlsx", sheet_name="Activity_Data")
ef = pd.read_excel("/content/drive/MyDrive/Data Vis/mock_carbon_data.xlsx", sheet_name="Emission_Factors")

# **Merge + Guardrail + Sanity Check**

In [ ]:
## Merge + Guardrail
merged = rawdata.merge(ef, on=["Source", "Unit"], how="left")
merged["CO2e_calc (t)"] = merged["Usage"] * merged["Emission_Factor (tCO2/unit)"]

missing = merged["Emission_Factor (tCO2/unit)"].isna().sum()
assert missing == 0, f"Missing emission factors for {missing} rows. Fix Emission_Factors Table."

In [7]:
## Validation / Sanity Check
merged[merged["Emission_Factor (tCO2/unit)"].isna()][["Source","Unit"]].drop_duplicates()
merged["diff_CO2e"] =merged["CO2e_calc (t)"] - merged["CO2e (t)"]
merged["diff_CO2e"].describe()

,diff_CO2e
count,48.000000
mean,0.000102
std,0.003120
min,-0.004560
25%,-0.002475
50%,0.000060
75%,0.002570
max,0.004980


# **Clean Dataset & Export**

In [ ]:
## clean dataset
clean_col = ["Date","Site/Dept","Source","Usage","Unit","CO2e_calc (t)"]
clean = merged[clean_col].copy().rename(columns={"CO2e_calc (t)":"CO2e (t)"})

In [10]:
## export data
clean.to_csv("/content/drive/MyDrive/Data Vis/cleaned_emissions.csv", index=False)

# **DataSet Management**

In [11]:
## Aggregation by month
monthly = (clean
           .assign(Date=pd.to_datetime(clean["Date"]))
           .groupby(pd.Grouper(key="Date", freq="MS"))["CO2e (t)"]
           .sum()
           .reset_index()
           .rename(columns={"Date":"Month"}))

In [13]:
## Aggregation by source
bysource=(clean
          .assign(Date=pd.to_datetime(clean["Date"]))
          .groupby([pd.Grouper(key="Date",freq ="MS"), "Source"]) ["CO2e (t)"]
          .sum()
          .reset_index()
          .rename(columns={"Date":"Month"}))